# ⚡ Intermédiaire | ⏱ 60 min | 🔑 Concepts : re, patterns, groups, findall, sub

# Expressions Régulières (Regex) en Python

## 🎯 Objectifs

- Comprendre la syntaxe des expressions régulières
- Maîtriser le module `re` de Python
- Utiliser les groupes de capture et groupes nommés
- Connaître les différences entre greedy et lazy matching
- Savoir chercher, extraire et remplacer du texte
- Éviter les pièges courants (catastrophic backtracking, etc.)

## 📋 Prérequis

- Python 3.8+
- Connaissances de base en chaînes de caractères
- Patience et persévérance (les regex peuvent être complexes !)

## 1. Module re : Pourquoi les regex ?

Les expressions régulières (regex) sont des patterns de recherche puissants pour :
- **Valider** des formats (email, téléphone, code postal)
- **Extraire** des informations (dates, URLs, numéros)
- **Remplacer** du texte avec des patterns complexes
- **Parser** des logs, fichiers de configuration, etc.

**Quand les utiliser ?**
- ✅ Patterns complexes que `str.find()` ne peut pas gérer
- ✅ Extraction de données structurées
- ✅ Validation de formats

**Quand les éviter ?**
- ❌ Parsing HTML/XML (utilisez BeautifulSoup, lxml)
- ❌ Recherche simple de chaîne (`in`, `str.find()` suffisent)
- ❌ Quand la lisibilité est critique (préférez du code explicite)

In [ ]:
import re

# Exemple simple : rechercher un pattern
texte = "Mon numéro est 06-12-34-56-78 et mon email est user@example.com"

# Rechercher un numéro de téléphone français
pattern_tel = r"\d{2}-\d{2}-\d{2}-\d{2}-\d{2}"
match = re.search(pattern_tel, texte)

if match:
    print(f"Téléphone trouvé : {match.group()}")
    print(f"Position : {match.start()} à {match.end()}")

# Rechercher un email
pattern_email = r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}"
match = re.search(pattern_email, texte)

if match:
    print(f"Email trouvé : {match.group()}")

## 2. Caractères spéciaux

| Caractère | Signification | Exemple |
|-----------|---------------|----------|
| `.` | N'importe quel caractère (sauf \n) | `a.c` → abc, a1c, a c |
| `^` | Début de ligne | `^Hello` → doit commencer par Hello |
| `$` | Fin de ligne | `world$` → doit finir par world |
| `*` | 0 ou plus | `ab*c` → ac, abc, abbc |
| `+` | 1 ou plus | `ab+c` → abc, abbc (pas ac) |
| `?` | 0 ou 1 (optionnel) | `ab?c` → ac, abc |
| `{n}` | Exactement n fois | `a{3}` → aaa |
| `{n,m}` | Entre n et m fois | `a{2,4}` → aa, aaa, aaaa |
| `[]` | Classe de caractères | `[abc]` → a, b ou c |
| `[^]` | Négation | `[^abc]` → tout sauf a, b, c |
| `|` | Alternative (OU) | `cat|dog` → cat ou dog |
| `()` | Groupe de capture | `(ab)+` → ab, abab |
| `\` | Échappement | `\.` → point littéral |

In [ ]:
# Exemples de caractères spéciaux

# Point : n'importe quel caractère
print("Point (.) :")
print(re.findall(r"a.c", "abc a1c a c axc"))  # ['abc', 'a1c', 'a c', 'axc']

# Début et fin de ligne
print("\nDébut (^) et fin ($) :")
print(re.search(r"^Hello", "Hello world"))  # Match
print(re.search(r"^Hello", "Say Hello"))    # None
print(re.search(r"world$", "Hello world"))  # Match

# Quantificateurs
print("\nQuantificateurs :")
print(re.findall(r"ab*c", "ac abc abbc abbbc"))    # * : 0 ou plus
print(re.findall(r"ab+c", "ac abc abbc abbbc"))    # + : 1 ou plus
print(re.findall(r"ab?c", "ac abc abbc"))          # ? : 0 ou 1
print(re.findall(r"a{3}", "a aa aaa aaaa"))        # {n} : exactement n
print(re.findall(r"a{2,3}", "a aa aaa aaaa"))      # {n,m} : entre n et m

# Classes de caractères
print("\nClasses de caractères :")
print(re.findall(r"[aeiou]", "hello world"))       # Voyelles
print(re.findall(r"[^aeiou]", "hello"))            # Tout sauf voyelles
print(re.findall(r"[0-9]", "abc123def456"))        # Chiffres
print(re.findall(r"[a-z]", "Hello World 123"))     # Lettres minuscules

# Alternative
print("\nAlternative (|) :")
print(re.findall(r"chat|chien|oiseau", "J'ai un chat et un chien"))

## 3. Classes de caractères prédéfinies

Python fournit des raccourcis pour les classes courantes :

| Classe | Équivalent | Description |
|--------|------------|-------------|
| `\d` | `[0-9]` | Chiffre |
| `\D` | `[^0-9]` | Non-chiffre |
| `\w` | `[a-zA-Z0-9_]` | Caractère de mot |
| `\W` | `[^a-zA-Z0-9_]` | Non-caractère de mot |
| `\s` | `[ \t\n\r\f\v]` | Espace blanc |
| `\S` | `[^ \t\n\r\f\v]` | Non-espace |
| `\b` | - | Frontière de mot |
| `\B` | - | Non-frontière de mot |

In [ ]:
texte = "Prix: 19.99€, Réf: ABC-123, Date: 2024-01-15"

# \d : chiffres
print("Chiffres (\\d) :")
print(re.findall(r"\d+", texte))  # ['19', '99', '123', '2024', '01', '15']

# \w : caractères alphanumériques
print("\nMots (\\w+) :")
print(re.findall(r"\w+", texte))

# \s : espaces blancs
print("\nEspaces (\\s) :")
print(re.findall(r"\s+", "Hello   world\t!\n"))

# \b : frontière de mot (très utile pour recherche exacte)
print("\nFrontière de mot (\\b) :")
texte2 = "cat cats caterpillar"
print(re.findall(r"\bcat\b", texte2))      # ['cat'] - exact
print(re.findall(r"cat", texte2))          # ['cat', 'cat', 'cat'] - tous

# Exemple pratique : extraire un prix
print("\nExtraire un prix :")
prix_pattern = r"\d+\.\d{2}€"
print(re.search(prix_pattern, texte).group())  # '19.99€'

## 4. Greedy vs Lazy (Avide vs Paresseux)

Par défaut, les quantificateurs (`*`, `+`, `?`, `{}`) sont **greedy** : ils consomment le plus de caractères possible.

Pour les rendre **lazy** (paresseux), on ajoute `?` après le quantificateur :
- `*?` : 0 ou plus (lazy)
- `+?` : 1 ou plus (lazy)
- `??` : 0 ou 1 (lazy)
- `{n,m}?` : entre n et m (lazy)

In [ ]:
html = "<div>Contenu 1</div><div>Contenu 2</div>"

# GREEDY : prend le maximum
greedy_pattern = r"<div>.*</div>"
match = re.search(greedy_pattern, html)
print("Greedy (.*) :")
print(f"  Résultat : {match.group()}")
print(f"  → Prend TOUT du premier <div> au dernier </div>\n")

# LAZY : prend le minimum
lazy_pattern = r"<div>.*?</div>"
matches = re.findall(lazy_pattern, html)
print("Lazy (.*?) :")
print(f"  Résultat : {matches}")
print(f"  → Prend chaque <div>...</div> séparément\n")

# Autre exemple
texte = "aaa bbb ccc"
print("Exemple sur 'aaa bbb ccc' :")
print(f"  Greedy (.+) : {re.findall(r'.+', texte)}")   # Toute la ligne
print(f"  Lazy (.+?) : {re.findall(r'.+?', texte)}")    # Caractère par caractère
print(f"  Lazy (.+?) avec espace : {re.findall(r'.+? ', texte)}")  # Jusqu'au premier espace

## 5. Fonctions du module re

### 5.1 re.match() vs re.search() vs re.findall()

In [ ]:
texte = "Python est formidable. Python rocks!"

# re.match() : vérifie si le pattern est au DÉBUT
print("re.match() :")
match = re.match(r"Python", texte)
print(f"  Au début : {match.group() if match else 'Pas de match'}")

match = re.match(r"formidable", texte)
print(f"  Au milieu : {match.group() if match else 'Pas de match'}")

# re.search() : cherche PARTOUT (première occurrence)
print("\nre.search() :")
search = re.search(r"formidable", texte)
print(f"  Résultat : {search.group() if search else 'Pas de match'}")
print(f"  Position : {search.start()}-{search.end()}")

# re.findall() : trouve TOUTES les occurrences (retourne une liste)
print("\nre.findall() :")
matches = re.findall(r"Python", texte)
print(f"  Résultats : {matches}")
print(f"  Nombre : {len(matches)}")

# re.finditer() : comme findall mais retourne des objets Match (itérateur)
print("\nre.finditer() :")
for match in re.finditer(r"Python", texte):
    print(f"  '{match.group()}' à la position {match.start()}")

### 5.2 re.sub() : Remplacement

In [ ]:
# re.sub(pattern, replacement, string) : remplace les occurrences

# Remplacement simple
texte = "J'adore Python. Python est génial!"
nouveau = re.sub(r"Python", "JavaScript", texte)
print(f"Simple : {nouveau}")

# Limiter le nombre de remplacements
nouveau = re.sub(r"Python", "Rust", texte, count=1)
print(f"Count=1 : {nouveau}")

# Remplacement avec fonction callback
def uppercase_match(match):
    return match.group().upper()

texte = "Les mots python, java, rust sont des langages"
nouveau = re.sub(r"\b(python|java|rust)\b", uppercase_match, texte)
print(f"\nCallback : {nouveau}")

# Remplacement avec backreferences (groupes)
texte = "Nom: Dupont, Prénom: Jean"
# \1 référence le premier groupe, \2 le deuxième
nouveau = re.sub(r"(\w+): (\w+)", r"\2 (\1)", texte)
print(f"\nBackreferences : {nouveau}")

# Nettoyer des espaces multiples
texte = "Trop    d'espaces   ici"
nouveau = re.sub(r"\s+", " ", texte)
print(f"\nNettoyer espaces : {nouveau}")

# Masquer des numéros de carte bancaire
cb = "Ma carte: 1234-5678-9012-3456"
nouveau = re.sub(r"\d{4}-\d{4}-\d{4}-(\d{4})", r"****-****-****-\1", cb)
print(f"\nMasquer CB : {nouveau}")

### 5.3 re.compile() : Optimisation

In [ ]:
# re.compile() : pré-compile le pattern pour réutilisation
# Utile quand on utilise le même pattern plusieurs fois

# Sans compilation (moins performant si répété)
texte = "email1@example.com, email2@test.fr, email3@company.org"
for _ in range(3):
    matches = re.findall(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}", texte)

# Avec compilation (plus performant)
email_pattern = re.compile(r"[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}")
for _ in range(3):
    matches = email_pattern.findall(texte)

print("Emails trouvés :", matches)

# L'objet compilé a les mêmes méthodes
print("\nMéthodes disponibles :")
print(f"  search : {email_pattern.search(texte).group()}")
print(f"  findall : {email_pattern.findall(texte)}")
print(f"  sub : {email_pattern.sub('[EMAIL]', texte)}")

# Avantage : pattern réutilisable et lisible
EMAIL_REGEX = re.compile(r""" 
    [a-zA-Z0-9._%+-]+   # Partie locale
    @                    # @
    [a-zA-Z0-9.-]+      # Domaine
    \.                   # Point
    [a-zA-Z]{2,}        # TLD
""", re.VERBOSE)  # re.VERBOSE permet les commentaires dans le pattern

print(f"\nAvec VERBOSE : {EMAIL_REGEX.findall(texte)}")

## 6. Groupes de capture

Les parenthèses `()` créent des **groupes de capture** qui permettent d'extraire des sous-parties du match.

In [ ]:
# Groupes simples
texte = "Date: 2024-01-15"
pattern = r"(\d{4})-(\d{2})-(\d{2})"
match = re.search(pattern, texte)

print("Groupes numérotés :")
print(f"  Match complet : {match.group(0)}")
print(f"  Année : {match.group(1)}")
print(f"  Mois : {match.group(2)}")
print(f"  Jour : {match.group(3)}")
print(f"  Tous les groupes : {match.groups()}")

# Groupes nommés : (?P<name>...)
pattern_nom = r"(?P<annee>\d{4})-(?P<mois>\d{2})-(?P<jour>\d{2})"
match = re.search(pattern_nom, texte)

print("\nGroupes nommés :")
print(f"  Année : {match.group('annee')}")
print(f"  Mois : {match.group('mois')}")
print(f"  Jour : {match.group('jour')}")
print(f"  Dict : {match.groupdict()}")

# Exemple pratique : parser une ligne de log
log = "[2024-01-15 14:32:45] ERROR: Connection failed"
pattern_log = r"\[(?P<date>[\d-]+) (?P<heure>[\d:]+)\] (?P<niveau>\w+): (?P<message>.*)"
match = re.search(pattern_log, log)

if match:
    print("\nParsing de log :")
    for key, value in match.groupdict().items():
        print(f"  {key:8} : {value}")

In [ ]:
# Groupes non-capturants : (?:...)
# Utile pour grouper sans capturer (performance)

texte = "http://example.com https://secure.example.com"

# Avec capture (2 groupes)
pattern_capture = r"(http|https)://([a-z.]+)"
matches = re.findall(pattern_capture, texte)
print("Avec capture :")
print(f"  Résultat : {matches}")  # [('http', 'example.com'), ('https', 'secure.example.com')]

# Sans capture du protocole (1 seul groupe)
pattern_non_capture = r"(?:http|https)://([a-z.]+)"
matches = re.findall(pattern_non_capture, texte)
print("\nSans capture (protocole) :")
print(f"  Résultat : {matches}")  # ['example.com', 'secure.example.com']

# Groupes conditionnels et autres features avancées
# Backreference : référencer un groupe précédent
texte_html = "<b>gras</b> et <i>italique</i> et <b>encore gras</b>"
pattern_tag = r"<(\w+)>.*?</\1>"  # \1 = référence au premier groupe
matches = re.findall(pattern_tag, texte_html)
print("\nBackreference (balises fermantes) :")
print(f"  Tags trouvés : {matches}")

## 7. Flags (options)

Les flags modifient le comportement du pattern :

| Flag | Constante | Description |
|------|-----------|-------------|
| `re.I` | `re.IGNORECASE` | Insensible à la casse |
| `re.M` | `re.MULTILINE` | ^ et $ matchent chaque ligne |
| `re.S` | `re.DOTALL` | . matche aussi \n |
| `re.X` | `re.VERBOSE` | Permet espaces et commentaires |
| `re.A` | `re.ASCII` | \w, \b, etc. ASCII seulement |
| `re.L` | `re.LOCALE` | Selon la locale système |

In [ ]:
# re.IGNORECASE : insensible à la casse
texte = "Python est un Langage FORMIDABLE"
print("re.IGNORECASE :")
print(f"  Sans flag : {re.findall(r'python', texte)}")
print(f"  Avec flag : {re.findall(r'python', texte, re.IGNORECASE)}")

# re.MULTILINE : ^ et $ pour chaque ligne
texte_multi = """ligne 1
ligne 2
ligne 3"""

print("\nre.MULTILINE :")
print(f"  Sans flag : {re.findall(r'^ligne', texte_multi)}")
print(f"  Avec flag : {re.findall(r'^ligne', texte_multi, re.MULTILINE)}")

# re.DOTALL : . matche aussi les retours à la ligne
html = "<div>\nContenu\nsur\nplusieurs\nlignes\n</div>"
print("\nre.DOTALL :")
print(f"  Sans flag : {re.search(r'<div>.*</div>', html)}")
print(f"  Avec flag : {re.search(r'<div>.*</div>', html, re.DOTALL).group()}")

# re.VERBOSE : permet commentaires et espaces
email_pattern = re.compile(r"""
    [a-zA-Z0-9._%+-]+   # Partie locale (avant @)
    @                    # Arobase
    [a-zA-Z0-9.-]+      # Nom de domaine
    \.                   # Point avant TLD
    [a-zA-Z]{2,}        # TLD (com, fr, org...)
""", re.VERBOSE)

print("\nre.VERBOSE :")
print(f"  Pattern lisible : {email_pattern.findall('contact@example.com')}")

# Combiner plusieurs flags avec |
pattern = re.compile(r'^python', re.IGNORECASE | re.MULTILINE)
texte = "Python\nest\nPYTHON"
print("\nFlags combinés (IGNORECASE | MULTILINE) :")
print(f"  Résultat : {pattern.findall(texte)}")

## 8. Raw strings : r""

**IMPORTANT** : Toujours utiliser des **raw strings** (`r"..."`) pour les regex en Python.

Pourquoi ? Les backslashes `\` ont une signification spéciale en Python ET dans les regex.

In [ ]:
# Problème avec les strings normales
# Pour matcher \d (chiffre en regex), il faut \\d en string normale

texte = "Code: 12345"

# ❌ MAUVAIS : string normale
# pattern = "\d+"  # Python interprète \d comme caractère d'échappement invalide
# Il faut doubler : "\\d+"
print("Sans raw string (à éviter) :")
print(f"  Pattern : '\\\\d+' → {re.findall('\\d+', texte)}")

# ✅ BON : raw string
print("\nAvec raw string (recommandé) :")
print(f"  Pattern : r'\\d+' → {re.findall(r'\d+', texte)}")

# Exemple plus complexe
chemin = r"C:\Users\Documents\file.txt"

# Pour matcher le backslash, il faut:
# - String normale : "\\\\" (4 backslashes!)
# - Raw string : r"\\" (2 backslashes)

print("\nMatcher un backslash :")
print(f"  Chemin : {chemin}")
print(f"  Parties (raw) : {re.split(r'\\\\', chemin)}")

# Tableau récapitulatif
print("\n" + "="*60)
print("Pourquoi utiliser r'' :")
print("="*60)
print("Pattern regex | String normale | Raw string")
print("-" * 60)
print(r"\d           | '\\d'         | r'\d'")
print(r"\w+          | '\\w+'        | r'\w+'")
print(r"\s           | '\\s'         | r'\s'")
print(r"\            | '\\\\'         | r'\\'")
print("="*60)
print("→ TOUJOURS utiliser r'' pour les regex !")

## 🚨 Pièges courants

### 1. Greedy par défaut

In [ ]:
# ❌ PROBLÈME : greedy consomme trop
html = "<p>Premier</p> du texte <p>Deuxième</p>"
mauvais = re.findall(r"<p>.*</p>", html)
print(f"Greedy (.*) : {mauvais}")
print("  → Prend TOUT du premier <p> au dernier </p>\n")

# ✅ SOLUTION : utiliser lazy (.*?)
bon = re.findall(r"<p>.*?</p>", html)
print(f"Lazy (.*?) : {bon}")
print("  → Prend chaque <p>...</p> séparément")

### 2. Oublier les raw strings r""

In [ ]:
texte = "Prix: 123.45€"

# ❌ MAUVAIS : oubli du r
# pattern = "\d+\.\d+"  # Erreur ou comportement inattendu
# Il faut: "\\d+\\.\\d+" (illisible)

# ✅ BON : raw string
pattern = r"\d+\.\d+"
print(f"Prix trouvé : {re.search(pattern, texte).group()}")

### 3. match() vs search()

In [ ]:
texte = "Le code est 12345"

# ❌ ERREUR : utiliser match() au lieu de search()
match = re.match(r"\d+", texte)
print(f"match() : {match}")
print("  → None car le texte ne COMMENCE PAS par un chiffre\n")

# ✅ BON : search() cherche partout
search = re.search(r"\d+", texte)
print(f"search() : {search.group()}")
print("  → Trouve le nombre n'importe où")

# Astuce : pour forcer un match complet, utilisez ^ et $
print("\nMatch complet avec ^ et $ :")
print(f"  '12345' : {re.match(r'^\d+$', '12345')}")
print(f"  'abc123' : {re.match(r'^\d+$', 'abc123')}")

### 4. Catastrophic Backtracking

Certains patterns peuvent causer des ralentissements exponentiels.

In [ ]:
# ❌ DANGEREUX : pattern avec backtracking exponentiel
# Pattern : (a+)+b
# Sur "aaaaaaaaaaaaaaaaaaaaX" (pas de 'b'), le moteur teste TOUTES les combinaisons
# → peut prendre des secondes/minutes !

# Ne pas exécuter ceci (trop lent) :
# bad_pattern = r"(a+)+b"
# re.search(bad_pattern, "a" * 20 + "X")

print("⚠️  Pattern dangereux : r'(a+)+b'")
print("Sur 'aaaa...aaaaX' (sans 'b'), le backtracking explose\n")

# ✅ SOLUTION : être plus spécifique
good_pattern = r"a+b"
print(f"✅ Pattern sûr : r'a+b'")
print(f"Résultat : {re.search(good_pattern, 'a' * 20 + 'X')}")
print("→ Échoue rapidement sans backtracking")

# Règle : éviter les quantificateurs imbriqués comme (a+)*, (a*)*, etc.

## 💪 Mini-exercices

### Exercice 1 : Valider un email

Créer une fonction qui valide une adresse email.

Règles simplifiées :
- Partie locale : lettres, chiffres, points, tirets, underscores
- @
- Domaine : lettres, chiffres, tirets, points
- TLD : 2-6 lettres

In [ ]:
import re

def valider_email(email: str) -> bool:
    """Valide une adresse email."""
    # TODO : écrire le pattern
    pattern = r""
    return re.match(pattern, email) is not None

# Tests
# tests = [
#     ("user@example.com", True),
#     ("user.name@example.co.uk", True),
#     ("user+tag@example.com", True),
#     ("invalid@", False),
#     ("@example.com", False),
#     ("user@.com", False),
# ]

# for email, attendu in tests:
#     resultat = valider_email(email)
#     statut = "✅" if resultat == attendu else "❌"
#     print(f"{statut} {email:30} → {resultat}")

### Solution Exercice 1

In [ ]:
import re

def valider_email(email: str) -> bool:
    """Valide une adresse email."""
    pattern = r"^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,6}$"
    return re.match(pattern, email) is not None

# Version avec groupes nommés (plus lisible)
def valider_email_verbose(email: str) -> bool:
    """Valide une adresse email (version verbose)."""
    pattern = re.compile(r"""
        ^                       # Début
        (?P<local>[a-zA-Z0-9._%+-]+)  # Partie locale
        @                       # Arobase
        (?P<domaine>[a-zA-Z0-9.-]+)   # Domaine
        \.                      # Point
        (?P<tld>[a-zA-Z]{2,6}) # TLD
        $                       # Fin
    """, re.VERBOSE)
    return pattern.match(email) is not None

# Tests
tests = [
    ("user@example.com", True),
    ("user.name@example.co.uk", True),
    ("user+tag@example.com", True),
    ("first.last@sub.domain.com", True),
    ("invalid@", False),
    ("@example.com", False),
    ("user@.com", False),
    ("user@domain", False),
]

print("Tests de validation d'email :\n")
for email, attendu in tests:
    resultat = valider_email(email)
    statut = "✅" if resultat == attendu else "❌"
    print(f"{statut} {email:30} → {resultat}")

### Exercice 2 : Extraire des numéros de téléphone

Extraire tous les numéros de téléphone français d'un texte.

Formats acceptés :
- 06 12 34 56 78
- 06-12-34-56-78
- 06.12.34.56.78
- 0612345678

In [ ]:
import re

def extraire_telephones(texte: str) -> list:
    """Extrait les numéros de téléphone français."""
    # TODO : écrire le pattern
    pattern = r""
    return re.findall(pattern, texte)

# Test
# texte = """
# Appelez-moi au 06 12 34 56 78 ou au 01-23-45-67-89.
# Vous pouvez aussi essayer 06.98.76.54.32 ou 0987654321.
# Le numéro invalide 123456 ne doit pas matcher.
# """

# telephones = extraire_telephones(texte)
# print(f"Numéros trouvés : {telephones}")

### Solution Exercice 2

In [ ]:
import re

def extraire_telephones(texte: str) -> list:
    """Extrait les numéros de téléphone français."""
    # Pattern : 0[1-9] suivi de 4 paires de 2 chiffres, séparés par espace/tiret/point ou rien
    pattern = r"\b0[1-9](?:[ .-]?\d{2}){4}\b"
    return re.findall(pattern, texte)

def extraire_telephones_normalises(texte: str) -> list:
    """Extrait et normalise les numéros de téléphone."""
    pattern = r"\b0[1-9](?:[ .-]?\d{2}){4}\b"
    telephones = re.findall(pattern, texte)
    # Normaliser en enlevant les séparateurs
    normalises = [re.sub(r"[ .-]", "", tel) for tel in telephones]
    return normalises

# Test
texte = """
Contactez-nous :
- Standard : 01 23 45 67 89
- Mobile : 06-12-34-56-78
- Fax : 01.98.76.54.32
- Direct : 0987654321
- Invalide : 123456 (trop court)
- Invalide : 00 12 34 56 78 (commence par 00)
"""

print("Numéros trouvés :")
telephones = extraire_telephones(texte)
for tel in telephones:
    print(f"  - {tel}")

print("\nNuméros normalisés :")
normalises = extraire_telephones_normalises(texte)
for tel in normalises:
    print(f"  - {tel}")

### Exercice 3 : Parser un fichier de log

Parser des lignes de log au format :
```
[2024-01-15 14:32:45] INFO: Application started
[2024-01-15 14:32:46] ERROR: Connection failed (code: 500)
```

Extraire : date, heure, niveau, message

In [ ]:
import re
from typing import Dict, List

def parser_log(ligne: str) -> Dict[str, str]:
    """Parse une ligne de log."""
    # TODO : écrire le pattern avec groupes nommés
    pattern = r""
    match = re.search(pattern, ligne)
    if match:
        return match.groupdict()
    return None

# Test
# logs = [
#     "[2024-01-15 14:32:45] INFO: Application started",
#     "[2024-01-15 14:32:46] ERROR: Connection failed (code: 500)",
#     "[2024-01-15 14:32:47] WARNING: High memory usage (85%)",
# ]

# for log in logs:
#     parsed = parser_log(log)
#     print(parsed)

### Solution Exercice 3

In [ ]:
import re
from typing import Dict, List, Optional
from collections import Counter

def parser_log(ligne: str) -> Optional[Dict[str, str]]:
    """Parse une ligne de log."""
    pattern = r"""\[
        (?P<date>\d{4}-\d{2}-\d{2})   # Date YYYY-MM-DD
        \s
        (?P<heure>\d{2}:\d{2}:\d{2})  # Heure HH:MM:SS
        \]\s
        (?P<niveau>\w+)                # Niveau (INFO, ERROR, etc.)
        :\s
        (?P<message>.+)                # Message (tout le reste)
    """
    match = re.search(pattern, ligne, re.VERBOSE)
    if match:
        return match.groupdict()
    return None

def analyser_logs(logs: List[str]) -> None:
    """Analyse une liste de logs."""
    parsed_logs = [parser_log(log) for log in logs if parser_log(log)]
    
    print(f"Total de logs parsés : {len(parsed_logs)}\n")
    
    # Statistiques par niveau
    niveaux = Counter(log['niveau'] for log in parsed_logs)
    print("Distribution par niveau :")
    for niveau, count in niveaux.most_common():
        print(f"  {niveau:10} : {count}")
    
    # Afficher les erreurs
    print("\nErreurs détectées :")
    erreurs = [log for log in parsed_logs if log['niveau'] == 'ERROR']
    for err in erreurs:
        print(f"  [{err['date']} {err['heure']}] {err['message']}")

# Test
logs = [
    "[2024-01-15 14:32:45] INFO: Application started",
    "[2024-01-15 14:32:46] ERROR: Connection failed (code: 500)",
    "[2024-01-15 14:32:47] WARNING: High memory usage (85%)",
    "[2024-01-15 14:32:48] INFO: Request processed in 120ms",
    "[2024-01-15 14:32:49] ERROR: Database timeout",
    "[2024-01-15 14:32:50] INFO: Cache cleared",
]

print("Parsing de logs :\n")
for log in logs[:3]:
    parsed = parser_log(log)
    if parsed:
        print(f"Date: {parsed['date']}")
        print(f"Heure: {parsed['heure']}")
        print(f"Niveau: {parsed['niveau']}")
        print(f"Message: {parsed['message']}")
        print()

print("="*60)
print("\nAnalyse globale :\n")
analyser_logs(logs)

## 📚 Ressources complémentaires

- [Documentation Python re](https://docs.python.org/3/library/re.html)
- [Regex101](https://regex101.com/) - Testeur en ligne avec explications
- [RegExr](https://regexr.com/) - Autre outil interactif
- [Regular-Expressions.info](https://www.regular-expressions.info/) - Guide complet
- [Debuggex](https://www.debuggex.com/) - Visualisation de regex
- [Python Regex Cheatsheet](https://www.datacamp.com/cheat-sheet/regular-expresso)

## 💡 Conseils

1. **Commencez simple** : construisez votre regex pas à pas
2. **Testez souvent** : utilisez des outils comme regex101.com
3. **Documentez** : utilisez `re.VERBOSE` pour les regex complexes
4. **Préférez la lisibilité** : parfois, du code Python simple > regex cryptique
5. **Évitez les regex pour HTML/XML** : utilisez des parsers dédiés
6. **Attention aux performances** : testez sur de vraies données